In [1]:
import numpy as np
import random

from keras.layers import Input,merge
from keras.layers.core import Reshape, Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Cropping2D, Convolution2D, Deconvolution2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam, SGD
import keras
from keras.models import Model

from tqdm import tqdm


Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)
/home/yiyuezhuo/anaconda3/lib/python3.6/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
from sklearn.externals import joblib
import numpy as np


In [3]:
Cropping2D?

In [4]:
import theano

In [5]:
i1 = i2 = 5
k1 = k2 = 3
#s1 = s2 = 1
#p1 = p2 = 2

In [7]:
output = theano.tensor.nnet.conv2d(
    input, filters, input_shape=(b, c2, i1, i2), filter_shape=(c1, c2, k1, k2),
    border_mode='full', subsample=(1, 1))


NameError: name 'filters' is not defined

In [9]:
theano.tensor.nnet.conv2d?

In [13]:
theano.tensor.

In [15]:
keras.layers.Deconvolution2D

In [20]:
def generator_block(input_node, input_shape, leaku_relu_alpha) :    
    channels, height, width = input_shape
    output_shape = (None, channels // 2, height * 2 + 2, width * 2 + 2)  
    
    block = Deconvolution2D(
        output_shape[1], 
        4, 4, 
        output_shape = output_shape, 
        border_mode='valid', 
        init='glorot_uniform', 
        subsample = (2, 2),
    )(input_node)
    block = Cropping2D(((1, 1), (1, 1)))(block)
    block = BatchNormalization(mode = 2)(block)
    block = LeakyReLU(leaku_relu_alpha)(block)

    return block


def build_generator(initial_size = (3, 3), channels_count = 32 * 32, leaky_relu_alpha = 0.2) :
    generator_input = Input(shape=[100])
    
    initial_height, initial_width = initial_size
    
    H = Dense(channels_count * initial_height * initial_width, init='glorot_normal')(generator_input)
    H = BatchNormalization(mode = 2)(H)
    H = LeakyReLU(0.2)(H)
    H = Reshape([channels_count, initial_height, initial_width] )(H) # nch

    # 4 layers result in x16 increased picture size, (3, 3) -> (48, 48)

    H = generator_block(H, (channels_count, initial_height, initial_width), leaky_relu_alpha)
    H = generator_block(H, (channels_count // 2 , initial_height * 2, initial_width * 2), leaky_relu_alpha)
    H = generator_block(H, (channels_count // 4 , initial_height * 4, initial_width * 4), leaky_relu_alpha)
    H = generator_block(H, (channels_count // 8 , initial_height * 8, initial_width * 8), leaky_relu_alpha)

    H = Convolution2D(channels_count // 32, 3, 3, border_mode='same', init='glorot_uniform')(H)
    H = BatchNormalization(mode = 2)(H)
    H = LeakyReLU(leaky_relu_alpha)(H)

    H = Convolution2D(3, 1, 1, border_mode='same', init='glorot_uniform')(H)
    generator_output = Activation('tanh')(H)

    generator = Model(generator_input, generator_output)
    generator_output_shape = (3, initial_height * 16, initial_width * 16)

    return generator, generator_input, generator_output_shape


In [21]:
generator, generator_input, generator_output_shape = build_generator()

In [22]:
generator.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 100)           0                                            
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 9216)          930816      input_3[0][0]                    
____________________________________________________________________________________________________
batchnormalization_3 (BatchNorma (None, 9216)          36864       dense_3[0][0]                    
____________________________________________________________________________________________________
leakyrelu_3 (LeakyReLU)          (None, 9216)          0           batchnormalization_3[0][0]       
___________________________________________________________________________________________